# Programming Assignment: 
## Готовим LDA по рецептам

Как вы уже знаете, в тематическом моделировании делается предположение о том, что для определения тематики порядок слов в документе не важен; об этом гласит гипотеза «мешка слов». Сегодня мы будем работать с несколько нестандартной для тематического моделирования коллекцией, которую можно назвать «мешком ингредиентов», потому что на состоит из рецептов блюд разных кухонь. Тематические модели ищут слова, которые часто вместе встречаются в документах, и составляют из них темы. Мы попробуем применить эту идею к рецептам и найти кулинарные «темы». Эта коллекция хороша тем, что не требует предобработки. Кроме того, эта задача достаточно наглядно иллюстрирует принцип работы тематических моделей.

Для выполнения заданий, помимо часто используемых в курсе библиотек, потребуются модули *json* и *gensim*. Первый входит в дистрибутив Anaconda, второй можно поставить командой 

*pip install gensim*

Построение модели занимает некоторое время. На ноутбуке с процессором Intel Core i7 и тактовой частотой 2400 МГц на построение одной модели уходит менее 10 минут.

### Загрузка данных

Коллекция дана в json-формате: для каждого рецепта известны его id, кухня (cuisine) и список ингредиентов, в него входящих. Загрузить данные можно с помощью модуля json (он входит в дистрибутив Anaconda):

In [5]:
import json

In [6]:
with open("recipes.json") as f:
    recipes = json.load(f)
recipes[0]

{'cuisine': 'greek',
 'id': 10259,
 'ingredients': ['romaine lettuce',
  'black olives',
  'grape tomatoes',
  'garlic',
  'pepper',
  'purple onion',
  'seasoning',
  'garbanzo beans',
  'feta cheese crumbles']}

### Составление корпуса

In [7]:
from gensim import corpora, models
import numpy as np

Наша коллекция небольшая, и целиком помещается в оперативную память. Gensim может работать с такими данными и не требует их сохранения на диск в специальном формате. Для этого коллекция должна быть представлена в виде списка списков, каждый внутренний список соответствует отдельному документу и состоит из его слов. Пример коллекции из двух документов: 

[["hello", "world"], ["programming", "in", "python"]]

Преобразуем наши данные в такой формат, а затем создадим объекты corpus и dictionary, с которыми будет работать модель.

In [8]:
texts = [recipe["ingredients"] for recipe in recipes]
dictionary = corpora.Dictionary(texts)   # составляем словарь
corpus = [dictionary.doc2bow(text) for text in texts]  # составляем корпус документов

In [10]:
texts[0]

['romaine lettuce',
 'black olives',
 'grape tomatoes',
 'garlic',
 'pepper',
 'purple onion',
 'seasoning',
 'garbanzo beans',
 'feta cheese crumbles']

In [14]:
corpus[0]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]

У объекта dictionary есть полезная переменная dictionary.token2id, позволяющая находить соответствие между ингредиентами и их индексами.

### Обучение модели
Вам может понадобиться [документация](https://radimrehurek.com/gensim/models/ldamodel.html) LDA в gensim.

__Задание 1.__ Обучите модель LDA с 40 темами, установив количество проходов по коллекции 5 и оставив остальные параметры по умолчанию. 


Затем вызовите метод модели *show_topics*, указав количество тем 40 и количество токенов 10, и сохраните результат (топы ингредиентов в темах) в отдельную переменную. Если при вызове метода *show_topics* указать параметр *formatted=True*, то топы ингредиентов будет удобно выводить на печать, если *formatted=False*, будет удобно работать со списком программно. Выведите топы на печать, рассмотрите темы, а затем ответьте на вопрос:

Сколько раз ингредиенты "salt", "sugar", "water", "mushrooms", "chicken", "eggs" встретились среди топов-10 всех 40 тем? При ответе __не нужно__ учитывать составные ингредиенты, например, "hot water".

Передайте 6 чисел в функцию save_answers1 и загрузите сгенерированный файл в форму.

У gensim нет возможности фиксировать случайное приближение через параметры метода, но библиотека использует numpy для инициализации матриц. Поэтому, по утверждению автора библиотеки, фиксировать случайное приближение нужно командой, которая написана в следующей ячейке. __Перед строкой кода с построением модели обязательно вставляйте указанную строку фиксации random.seed.__

In [15]:
np.random.seed(76543)
# здесь код для построения модели:
%time model = models.ldamodel.LdaModel(corpus, num_topics=40, id2word=dictionary, passes=5)

CPU times: user 6min 45s, sys: 224 ms, total: 6min 45s
Wall time: 6min 45s


In [16]:
topics = model.show_topics(num_topics=40, formatted=False)

In [17]:
model.show_topics(num_topics=40, formatted=True)

[(0,
  '0.084*"garlic cloves" + 0.068*"chopped onion" + 0.066*"cooking spray" + 0.063*"salt" + 0.043*"water" + 0.041*"fat free less sodium chicken broth" + 0.040*"ground red pepper" + 0.038*"sliced green onions" + 0.027*"olive oil" + 0.026*"black pepper"'),
 (1,
  '0.082*"corn kernels" + 0.070*"diced onions" + 0.059*"tortillas" + 0.057*"salt and ground black pepper" + 0.047*"1% low-fat milk" + 0.040*"sliced black olives" + 0.039*"cream cheese, soften" + 0.038*"vegetable stock" + 0.038*"chopped fresh chives" + 0.035*"lard"'),
 (2,
  '0.087*"olive oil" + 0.059*"fresh basil" + 0.049*"salt" + 0.047*"fresh parsley" + 0.046*"crushed red pepper" + 0.041*"grated parmesan cheese" + 0.041*"garlic cloves" + 0.030*"cherry tomatoes" + 0.027*"butter" + 0.027*"low salt chicken broth"'),
 (3,
  '0.074*"bacon" + 0.072*"salt" + 0.061*"red pepper flakes" + 0.049*"ground black pepper" + 0.047*"garlic" + 0.042*"pasta" + 0.038*"onions" + 0.035*"olive oil" + 0.035*"chicken thighs" + 0.030*"fresh dill"'),
 (4

In [18]:
ingredients = ["salt", "sugar", "water", "mushrooms", "chicken", "eggs"]

result = dict(zip(ingredients, [0] * 6))

for ingr in ingredients:
    for topic in topics:
        for top_word in topic[1]:
            if top_word[0] == ingr:
                result[ingr] += 1

result

{'chicken': 1, 'eggs': 2, 'mushrooms': 0, 'salt': 21, 'sugar': 9, 'water': 10}

In [19]:
values = [result.get(ingr) for ingr in ingredients]
values

[21, 9, 10, 0, 1, 2]

In [19]:
def save_answers1(c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs):
    with open("cooking_LDA_pa_task1.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs]]))

In [21]:
save_answers1(values[0], values[1], values[2], values[3], values[4], values[5])

### Фильтрация словаря
В топах тем гораздо чаще встречаются первые три рассмотренных ингредиента, чем последние три. При этом наличие в рецепте курицы, яиц и грибов яснее дает понять, что мы будем готовить, чем наличие соли, сахара и воды. Таким образом, даже в рецептах есть слова, часто встречающиеся в текстах и не несущие смысловой нагрузки, и поэтому их не желательно видеть в темах. Наиболее простой прием борьбы с такими фоновыми элементами — фильтрация словаря по частоте. Обычно словарь фильтруют с двух сторон: убирают очень редкие слова (в целях экономии памяти) и очень частые слова (в целях повышения интерпретируемости тем). Мы уберем только частые слова.

In [108]:
import copy
dictionary_2 = copy.deepcopy(dictionary)

__Задание 2.__ У объекта dictionary2 есть переменная *dfs* — это словарь, ключами которого являются id токена, а элементами — число раз, сколько слово встретилось во всей коллекции. Сохраните в отдельный список ингредиенты, которые встретились в коллекции больше 4000 раз. Вызовите метод словаря *filter_tokens*, подав в качестве первого аргумента полученный список популярных ингредиентов. Вычислите две величины: dict_size_before и dict_size_after — размер словаря до и после фильтрации.

Затем, используя новый словарь, создайте новый корпус документов, corpus2, по аналогии с тем, как это сделано в начале ноутбука. Вычислите две величины: corpus_size_before и corpus_size_after — суммарное количество ингредиентов в корпусе (для каждого документа вычислите число различных ингредиентов в нем и просуммируйте по всем документам) до и после фильтрации.

Передайте величины dict_size_before, dict_size_after, corpus_size_before, corpus_size_after в функцию save_answers2 и загрузите сгенерированный файл в форму.

In [109]:
dict_size_before = len(dictionary.dfs)

In [112]:
extra_ind_words = [key for key in dictionary_2.dfs.keys() if dictionary_2.dfs[key] > 4000]
extra_ind_words

[3, 5, 11, 15, 18, 20, 29, 44, 52, 59, 104, 114]

In [113]:
dictionary_2.filter_tokens(extra_ind_words)
dict_size_after = len(dictionary_2.dfs)
print(dict_size_before)
print(dict_size_after)

6714
6702


In [114]:
corpus_2 = [dictionary_2.doc2bow(text) for text in texts]
corpus_size_before = 0
corpus_size_after = 0

for c in corpus:
    corpus_size_before += len(c)

for c in corpus_2:
    corpus_size_after += len(c)
    
print(corpus_size_before)
print(corpus_size_after)

428249
343665


In [87]:
def save_answers2(dict_size_before, dict_size_after, corpus_size_before, corpus_size_after):
    with open("cooking_LDA_pa_task2.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [dict_size_before, dict_size_after, corpus_size_before, corpus_size_after]]))

In [88]:
save_answers2(dict_size_before, dict_size_after, corpus_size_before, corpus_size_after)

### Сравнение когерентностей
__Задание 3.__ Постройте еще одну модель по корпусу corpus2 и словарю dictionary2, остальные параметры оставьте такими же, как при первом построении модели. Сохраните новую модель в другую переменную (не перезаписывайте предыдущую модель). Не забудьте про фиксирование seed!

Затем воспользуйтесь методом *top_topics* модели, чтобы вычислить ее когерентность. Передайте в качестве аргумента соответствующий модели корпус. Метод вернет список кортежей (топ токенов, когерентность), отсортированных по убыванию последней. Вычислите среднюю по всем темам когерентность для каждой из двух моделей и передайте в функцию save_answers3. 

In [98]:
np.random.seed(76543)
# здесь код для построения модели:
%time second_model = models.ldamodel.LdaModel(corpus_2, num_topics=40, id2word=dictionary_2, passes=5)

CPU times: user 5min 50s, sys: 88 ms, total: 5min 50s
Wall time: 5min 50s


In [99]:
coh  = model.top_topics(corpus)
coh_2 = second_model.top_topics(corpus_2)
print(coh[:3])
print(coh_2[:3])

[([(0.07470618, 'sour cream'), (0.04646297, 'chili powder'), (0.045566555, 'salsa'), (0.04498307, 'flour tortillas'), (0.03975233, 'shredded cheddar cheese'), (0.039275248, 'black beans'), (0.037687555, 'corn tortillas'), (0.02913432, 'cilantro'), (0.028640373, 'ground cumin'), (0.027310649, 'onions'), (0.026456006, 'salt'), (0.02404393, 'green onions'), (0.02113124, 'avocado'), (0.020965634, 'cumin'), (0.01812089, 'tomatoes'), (0.017340321, 'jalapeno chilies'), (0.01663795, 'olive oil'), (0.0152053675, 'corn'), (0.015093604, 'enchilada sauce'), (0.014856199, 'garlic')], -2.189890114250082), ([(0.08774398, 'soy sauce'), (0.054586165, 'sesame oil'), (0.0461808, 'scallions'), (0.044415973, 'green onions'), (0.03923337, 'corn starch'), (0.03877022, 'rice vinegar'), (0.036451384, 'sugar'), (0.032991447, 'garlic'), (0.031374607, 'vegetable oil'), (0.02654085, 'fresh ginger'), (0.025166744, 'water'), (0.025108837, 'ginger'), (0.022585064, 'sesame seeds'), (0.020602163, 'salt'), (0.020497594,

In [102]:
nums = [c[1] for c in coh]
nums_2 = [c[1] for c in coh_2]
print(nums)
print(nums_2)

[-2.189890114250082, -2.3812265876644365, -2.4231013694600465, -2.6404301145967835, -2.7763520807543136, -2.8454041504995753, -3.000492762534722, -3.018488360769831, -3.132109412565028, -3.145299180624706, -3.157187161721497, -3.2629401970283376, -3.2801800797009295, -3.3570320376187595, -3.386796198092065, -3.4862641247234962, -3.737235228840055, -3.7815922439619656, -3.820541568356231, -4.019589387903069, -4.075628937421615, -4.436982949171743, -4.608996583865164, -4.8115250226059585, -4.838593993196258, -4.854696204845901, -5.875580559035723, -6.072235653328083, -6.199675161075947, -7.795644693624812, -8.668198601897082, -9.387545119830756, -10.961139042754104, -11.089534095841602, -11.126384474736245, -11.497323165418242, -13.22178490430017, -14.652786676785686, -16.50949443870023, -17.297955995958]
[-2.6820476724476943, -2.7313580184867887, -2.7682497157167134, -2.918380327259511, -3.1533236398379736, -3.188420639293828, -3.7820034230857225, -3.8278936211965666, -3.860168161324259

In [103]:
def save_answers3(coherence, coherence2):
    with open("cooking_LDA_pa_task3.txt", "w") as fout:
        fout.write(" ".join(["%3f"%el for el in [coherence, coherence2]]))

In [104]:
save_answers3(np.mean(nums), np.mean(nums_2))

Считается, что когерентность хорошо соотносится с человеческими оценками интерпретируемости тем. Поэтому на больших текстовых коллекциях когерентность обычно повышается, если убрать фоновую лексику. Однако в нашем случае этого не произошло. 

### Изучение влияния гиперпараметра alpha

В этом разделе мы будем работать со второй моделью, то есть той, которая построена по сокращенному корпусу. 

Пока что мы посмотрели только на матрицу темы-слова, теперь давайте посмотрим на матрицу темы-документы. Выведите темы для нулевого (или любого другого) документа из корпуса, воспользовавшись методом *get_document_topics* второй модели:

In [105]:
second_model.get_document_topics(corpus_2[0])

[(25, 0.128125), (30, 0.13281512), (31, 0.6234349)]

Также выведите содержимое переменной *.alpha* второй модели:

In [106]:
second_model.alpha

array([0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
       0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
       0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
       0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
       0.025, 0.025, 0.025, 0.025], dtype=float32)

У вас должно получиться, что документ характеризуется небольшим числом тем. Попробуем поменять гиперпараметр alpha, задающий априорное распределение Дирихле для распределений тем в документах.

__Задание 4.__ Обучите третью модель: используйте сокращенный корпус (corpus2 и dictionary2) и установите параметр __alpha=1__, passes=5. Не забудьте про фиксацию seed! Выведите темы новой модели для нулевого документа; должно получиться, что распределение над множеством тем практически равномерное. Чтобы убедиться в том, что во второй модели документы описываются гораздо более разреженными распределениями, чем в третьей, посчитайте суммарное количество элементов, __превосходящих 0.01__, в матрицах темы-документы обеих моделей. Другими словами, запросите темы  модели для каждого документа с параметром *minimum_probability=0.01* и просуммируйте число элементов в получаемых массивах. Передайте две суммы (сначала для модели с alpha по умолчанию, затем для модели в alpha=1) в функцию save_answers4.

In [115]:
np.random.seed(76543)
# здесь код для построения модели:
%time third_model = models.ldamodel.LdaModel(corpus_2, num_topics=40, id2word=dictionary_2, passes=5, alpha=1)

CPU times: user 18min 9s, sys: 600 ms, total: 18min 9s
Wall time: 18min 10s


In [124]:
third_model.get_document_topics(corpus_2[0])

[(0, 0.021348575),
 (1, 0.021298638),
 (2, 0.021276595),
 (3, 0.021338549),
 (4, 0.02128087),
 (5, 0.021382393),
 (6, 0.021299792),
 (7, 0.021277202),
 (8, 0.021449137),
 (9, 0.021319209),
 (10, 0.02164358),
 (11, 0.042108793),
 (12, 0.021276595),
 (13, 0.046975877),
 (14, 0.021555642),
 (15, 0.021562053),
 (16, 0.021459104),
 (17, 0.021820657),
 (18, 0.021276603),
 (19, 0.021417111),
 (20, 0.024527254),
 (21, 0.021288445),
 (22, 0.021276595),
 (23, 0.021283576),
 (24, 0.021810684),
 (25, 0.021577844),
 (26, 0.021523274),
 (27, 0.02134925),
 (28, 0.021500375),
 (29, 0.02132352),
 (30, 0.042586245),
 (31, 0.0710489),
 (32, 0.021418689),
 (33, 0.021310415),
 (34, 0.02129087),
 (35, 0.021343203),
 (36, 0.021288779),
 (37, 0.021294925),
 (38, 0.021279102),
 (39, 0.045311056)]

In [126]:
third_model.alpha

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1.], dtype=float32)

In [117]:
def save_answers4(count_model2, count_model3):
    with open("cooking_LDA_pa_task4.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [count_model2, count_model3]]))

In [119]:
distr_1 = second_model.get_document_topics(corpus_2, minimum_probability=0.01)
distr_2 = third_model.get_document_topics(corpus_2, minimum_probability=0.01)

In [ ]:
size_1 = 0
for x in distr_1:
    size_1 += len(x)
    
size_2 = 0
for x in distr_2:
    size_2 += len(x)
    
save_answers4(size_1, size_2)

Таким образом, гиперпараметр __alpha__ влияет на разреженность распределений тем в документах. Аналогично гиперпараметр __eta__ влияет на разреженность распределений слов в темах.

### LDA как способ понижения размерности
Иногда, распределения над темами, найденные с помощью LDA, добавляют в матрицу объекты-признаки как дополнительные, семантические, признаки, и это может улучшить качество решения задачи. Для простоты давайте просто обучим классификатор рецептов на кухни на признаках, полученных из LDA, и измерим точность (accuracy).

__Задание 5.__ Используйте модель, построенную по сокращенной выборке с alpha по умолчанию (вторую модель). Составьте матрицу $\Theta = p(t|d)$ вероятностей тем в документах; вы можете использовать тот же метод get_document_topics, а также вектор правильных ответов y (в том же порядке, в котором рецепты идут в переменной recipes). Создайте объект RandomForestClassifier со 100 деревьями, с помощью функции cross_val_score вычислите среднюю accuracy по трем фолдам (перемешивать данные не нужно) и передайте в функцию save_answers5.

In [132]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score

/home/pik/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [127]:
topics = second_model.get_document_topics(corpus_2, minimum_probability=0)
topics[2]

[(0, 0.0025),
 (1, 0.0025),
 (2, 0.0025),
 (3, 0.0025),
 (4, 0.0025),
 (5, 0.0025),
 (6, 0.0025),
 (7, 0.0025),
 (8, 0.0025),
 (9, 0.0025),
 (10, 0.0025),
 (11, 0.10438007),
 (12, 0.0025),
 (13, 0.0025),
 (14, 0.0025),
 (15, 0.1025),
 (16, 0.0025),
 (17, 0.12853052),
 (18, 0.0025),
 (19, 0.0025),
 (20, 0.0025),
 (21, 0.1025),
 (22, 0.16307384),
 (23, 0.21120591),
 (24, 0.0025),
 (25, 0.0025),
 (26, 0.0025),
 (27, 0.0025),
 (28, 0.0025),
 (29, 0.10530964),
 (30, 0.0025),
 (31, 0.0025),
 (32, 0.0025),
 (33, 0.0025),
 (34, 0.0025),
 (35, 0.0025),
 (36, 0.0025),
 (37, 0.0025),
 (38, 0.0025),
 (39, 0.0025)]

In [128]:
p = np.zeros([len(topics), 40])
for i, topic in enumerate(topics):
    for t in topic:
        p[i, t[0]] = t[1]   

In [129]:
y = [recipe["cuisine"] for recipe in recipes]
y[:5]

['greek', 'southern_us', 'filipino', 'indian', 'indian']

In [130]:
def save_answers5(accuracy):
     with open("cooking_LDA_pa_task5.txt", "w") as fout:
        fout.write(str(accuracy))

In [133]:
score = cross_val_score(RandomForestClassifier(n_estimators=100), p, y)
score[:3]

array([0.55266531, 0.54468663, 0.55999094])

In [134]:
save_answers5(score.mean())

Для такого большого количества классов это неплохая точность. Вы можете попроовать обучать RandomForest на исходной матрице частот слов, имеющей значительно большую размерность, и увидеть, что accuracy увеличивается на 10–15%. Таким образом, LDA собрал не всю, но достаточно большую часть информации из выборки, в матрице низкого ранга.

### LDA — вероятностная модель
Матричное разложение, использующееся в LDA, интерпретируется как следующий процесс генерации документов.

Для документа $d$ длины $n_d$:
1. Из априорного распределения Дирихле с параметром alpha сгенерировать распределение над множеством тем: $\theta_d \sim Dirichlet(\alpha)$
1. Для каждого слова $w = 1, \dots, n_d$:
    1. Сгенерировать тему из дискретного распределения $t \sim \theta_{d}$
    1. Сгенерировать слово из дискретного распределения $w \sim \phi_{t}$.
    
Подробнее об этом в [Википедии](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation).

В контексте нашей задачи получается, что, используя данный генеративный процесс, можно создавать новые рецепты. Вы можете передать в функцию модель и число ингредиентов и сгенерировать рецепт :)

In [135]:
def generate_recipe(model, num_ingredients):
    theta = np.random.dirichlet(model.alpha)
    for i in range(num_ingredients):
        t = np.random.choice(np.arange(model.num_topics), p=theta)
        topic = model.show_topic(t, topn=model.num_terms)
        topic_distr = [x[1] for x in topic]
        terms = [x[0] for x in topic]
        w = np.random.choice(terms, p=topic_distr)
        print(w)

In [ ]:
generate_recipe(model, 5)

### Интерпретация построенной модели
Вы можете рассмотреть топы ингредиентов каждой темы. Большиснтво тем сами по себе похожи на рецепты; в некоторых собираются продукты одного вида, например, свежие фрукты или разные виды сыра.

Попробуем эмпирически соотнести наши темы с национальными кухнями (cuisine). Построим матрицу $A$ размера темы $x$ кухни, ее элементы $a_{tc}$ — суммы $p(t|d)$ по всем документам $d$, которые отнесены к кухне $c$. Нормируем матрицу на частоты рецептов по разным кухням, чтобы избежать дисбаланса между кухнями. Следующая функция получает на вход объект модели, объект корпуса и исходные данные и возвращает нормированную матрицу $A$. Ее удобно визуализировать с помощью seaborn.

In [137]:
import pandas
import seaborn
from matplotlib import pyplot as plt
%matplotlib inline

In [138]:
def compute_topic_cuisine_matrix(model, corpus, recipes):
    # составляем вектор целевых признаков
    targets = list(set([recipe["cuisine"] for recipe in recipes]))
    # составляем матрицу
    tc_matrix = pandas.DataFrame(data=np.zeros((model.num_topics, len(targets))), columns=targets)
    for recipe, bow in zip(recipes, corpus):
        recipe_topic = model.get_document_topics(bow)
        for t, prob in recipe_topic:
            tc_matrix[recipe["cuisine"]][t] += prob
    # нормируем матрицу
    target_sums = pandas.DataFrame(data=np.zeros((1, len(targets))), columns=targets)
    for recipe in recipes:
        target_sums[recipe["cuisine"]] += 1
    return pandas.DataFrame(tc_matrix.values/target_sums.values, columns=tc_matrix.columns)

In [139]:
def plot_matrix(tc_matrix):
    plt.figure(figsize=(10, 10))
    seaborn.heatmap(tc_matrix, square=True)

In [ ]:
# Визуализируйте матрицу
matrix = compute_topic_cuisine_matrix(model, corpus, recipes)
plot_matrix(matrix)

Чем темнее квадрат в матрице, тем больше связь этой темы с данной кухней. Мы видим, что у нас есть темы, которые связаны с несколькими кухнями. Такие темы показывают набор ингредиентов, которые популярны в кухнях нескольких народов, то есть указывают на схожесть кухонь этих народов. Некоторые темы распределены по всем кухням равномерно, они показывают наборы продуктов, которые часто используются в кулинарии всех стран. 

Жаль, что в датасете нет названий рецептов, иначе темы было бы проще интерпретировать...

### Заключение
В этом задании вы построили несколько моделей LDA, посмотрели, на что влияют гиперпараметры модели и как можно использовать построенную модель. 